In [62]:
from PIL import Image
import numpy as np
import cv2
import random
from skimage.metrics import peak_signal_noise_ratio as psnr


def calculate_ber(bin_file1, bin_file2):
    with open(bin_file1, 'rb') as file1, open(bin_file2, 'rb') as file2:
        data1 = file1.read()
        data2 = file2.read()
    
    # Calculate the number of bit errors
    bit_errors = sum(bin(int(byte1) ^ int(byte2)).count('1') for byte1, byte2 in zip(data1, data2))
    
    # Calculate the bit error rate
    ber = bit_errors / (8 * len(data1))
    return ber, bit_errors, len(data1)*8



def compress_image(input_path, output_path, quality=85):
    # Open the image
    img = Image.open(input_path)
    
    # Convert image to RGB (if not already in RGB mode)
    if img.mode != "RGB":
        img = img.convert("RGB")
    
    # Save image in JPEG format with the specified quality
    img.save(output_path, "JPEG", quality=quality)
    print(f"Compressed image saved at: {output_path}")


def convert_img_to_bin(input_jpg_path, output_bin_path):
    """
    Convert a .jpg file to a .bin file.
    
    Parameters:
        input_jpg_path (str): Path to the input .jpg file.
        output_bin_path (str): Path to save the .bin file.
    """
    try:
        # Open the input .jpg file in binary read mode
        with open(input_jpg_path, 'rb') as jpg_file:
            # Read the binary content
            jpg_data = jpg_file.read()
        
        # Open the output .bin file in binary write mode
        with open(output_bin_path, 'wb') as bin_file:
            # Write the binary content to the .bin file
            bin_file.write(jpg_data)
        
        print(f"File successfully converted to .bin format and saved at {output_bin_path}.")
    except Exception as e:
        print(f"An error occurred: {e}")


def convert_bin_to_img(input_bin_path, output_jpg_path):
    """
    Convert a .bin file back to a .jpg file.
    
    Parameters:
        input_bin_path (str): Path to the input .bin file.
        output_jpg_path (str): Path to save the .jpg file.
    """
    try:
        # Open the input .bin file in binary read mode
        with open(input_bin_path, 'rb') as bin_file:
            # Read the binary content
            binary_data = bin_file.read()
        
        # Open the output .jpg file in binary write mode
        with open(output_jpg_path, 'wb') as jpg_file:
            # Write the binary content to the .jpg file
            jpg_file.write(binary_data)
        
        print(f"File successfully converted back to .jpg format and saved at {output_jpg_path}.")
    except Exception as e:
        print(f"An error occurred: {e}")



# Function to introduce random bit errors to the binary file
def introduce_bit_errors(binary_file_path, output_binary_file_path, ber=0.2):
    with open(binary_file_path, 'rb') as binary_file:
        binary_data = bytearray(binary_file.read())

    for i in range(len(binary_data)):
        for bit in range(8):  # Each byte has 8 bits
            if random.random() < ber:  # With probability equal to ber
                binary_data[i] ^= (1 << bit)  # Flip the bit

    with open(output_binary_file_path, 'wb') as output_file:
        output_file.write(binary_data)


def calc_psnr(image_path1, image_path2):
    """
    Calculate the PSNR between two images.
    
    Parameters:
        image_path1 (str): Path to the first image.
        image_path2 (str): Path to the second image.
    
    Returns:
        float: PSNR value in dB.
    """
    # Load the two images
    image1 = cv2.imread(image_path1)
    image2 = cv2.imread(image_path2)
    
    # Check if images are loaded successfully
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid or the images could not be loaded.")
    
    # Ensure the images have the same dimensions
    if image1.shape != image2.shape:
        raise ValueError("Images must have the same dimensions to calculate PSNR.")
    
    # Calculate PSNR using skimage
    psnr_value = psnr(image1, image2, data_range=255)
    return psnr_value



In [64]:
compress_image('kodim05.png', 'compressed_kodim5_quality30.jpg', quality=30)
convert_img_to_bin('compressed_kodim5_quality30.jpg', 'compressed_kodim5.bin')
introduce_bit_errors('compressed_kodim5.bin', 'corrupted_kodim5.bin', ber=0.0001)
convert_bin_to_img('corrupted_kodim5.bin', 'reconstructed_kodim5.jpg')
calc_psnr('kodim05.png', 'reconstructed_kodim5.jpg')

Compressed image saved at: Compressed_Kodim5_quality30.jpg
File successfully converted to .bin format and saved at compressed_Kodim5.bin.
File successfully converted back to .jpg format and saved at reconstructed_kodim5.jpg.


5.0353710360939505

In [75]:
introduce_bit_errors('encoded_feature_rayleigh_image24.bin', 'corrupted_features_rayleigh_image24.bin', ber=0.05)


In [73]:
ber, errors, bits = calculate_ber('encoded_feature_rayleigh_image24.bin', 'corrupted_feature_rayleigh_image24.bin')
print(f"Bit Error Rate: {ber:.2e}")
print(f"Number of bit errors: {errors}")
print(f"Total number of bits: {bits}")

Bit Error Rate: 9.90e-03
Number of bit errors: 3892
Total number of bits: 393216


In [76]:
calc_psnr('kodim05.png', 'reconstructed_original_example.jpg')

27.495655188889963

In [81]:
convert_img_to_bin('kodim05.png', 'compressed_kodim5_png.bin')
introduce_bit_errors('compressed_kodim5_png.bin', 'corrupted_kodim5_png.bin', ber=0.000001)
convert_bin_to_img('corrupted_kodim5_png.bin', 'reconstructed_kodim5_png.png')

File successfully converted to .bin format and saved at compressed_kodim5_png.bin.
File successfully converted back to .jpg format and saved at reconstructed_kodim5_png.png.
